In [2]:
import cv2
import numpy as np

In [3]:
# Initilizing the ORB Feature Detector

Min_Matches = 20
detector = cv2.ORB_create(nfeatures=3000)

In [4]:
# Preparing the FLANN Based matcher

index_params = dict(algorithm = 1, trees = 3)
search_params = dict(checks = 100)
flann = cv2.FlannBasedMatcher(index_params, search_params)

In [5]:
# Function for loading input image and keypoints

def load_input():
    
    input_image = cv2.imread(r'D:\Misc\Extra\one_piece_58.jpg')
    input_image = cv2.resize(input_image, (400, 600), interpolation = cv2.INTER_AREA)
    gray_image = cv2.cvtColor(input_image, cv2.COLOR_BGR2GRAY)
    
    # Find keypoints with ORB instance
    keypoints, descriptors = detector.detectAndCompute(gray_image, None)
    
    return gray_image, keypoints, descriptors

In [6]:
# Function for computing matches between trained model and descriptors from live feed

def compute_matches(descriptors_input, descriptors_output):
    
    if(len(descriptors_input)!=0 and len(descriptors_output)!=0):
        matches = flann.knnMatch(np.asarray(descriptors_input, np.float32), np.asarray(descriptors_output, np.float32), k=2)
        good = []
        for m, n in matches:
            if m.distance < 0.69 * n.distance:
                good.append([m])
        return good
    
    else:
        return None

In [7]:
# Main function

if __name__=='__main__':
    
    input_image, input_keypoints, input_descriptors = load_input()
    
    cap = cv2.VideoCapture(0)
    ret, frame = cap.read()
    
    while(ret==True):
        ret, frame = cap.read()
        
        if(len(input_keypoints) < Min_Matches):
            continue
        
        frame = cv2.resize(frame, (900,600))
        frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        
        output_keypoints, output_descriptors = detector.detectAndCompute(frame_gray, None)
        matches = compute_matches(input_descriptors, output_descriptors)
        
        if(matches==None):
            cv2.imshow('Final output', frame)
        
        else:
            output_final = cv2.drawMatchesKnn(input_image, input_keypoints, frame, output_keypoints, matches, None, 
                            flags = cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
            cv2.imshow('Final output', output_final)
        
        
        if(cv2.waitKey(10)==27):
            break